<a href="https://colab.research.google.com/github/enderunknown222/MA/blob/ver0.4/MA4550_Project_ver0.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import numpy as np
import pandas as pd
import tensorflow as tf
import io
import torch
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
from google.colab import files
data = pd.read_csv('https://raw.githubusercontent.com/enderunknown222/MA/Ver0.1/data.csv')
data2 = pd.read_csv('https://raw.githubusercontent.com/enderunknown222/MA/main/data2.csv')
coviddata = pd.read_csv('https://raw.githubusercontent.com/enderunknown222/MA/ver0.4/covid_data.csv')

In [26]:
#test data size
x=190
#Test data leaderboard
results = data.iloc[0:x,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(x):
  for j in ['Home','Away']:
    for k in range(20):
      if results[j][i]==Name[k]:
        results[j][i]=k
results
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(x):
  if results['HScore'][i]>results['AScore'][i]:
    leaderboard['Win'][results['Home'][i]]+=1
    leaderboard['Pts'][results['Home'][i]]+=3
    leaderboard['Loss'][results['Away'][i]]+=1
    leaderboard['Pts'][results['Away'][i]]+=0
  if results['HScore'][i]==results['AScore'][i]:
    leaderboard['Draw'][results['Home'][i]]+=1
    leaderboard['Pts'][results['Home'][i]]+=1
    leaderboard['Draw'][results['Away'][i]]+=1
    leaderboard['Pts'][results['Away'][i]]+=1
  if results['HScore'][i]<results['AScore'][i]:
    leaderboard['Loss'][results['Home'][i]]+=1
    leaderboard['Pts'][results['Home'][i]]+=0
    leaderboard['Win'][results['Away'][i]]+=1
    leaderboard['Pts'][results['Away'][i]]+=3
leaderboard.sort_values(by=['Pts'],inplace=True, ascending=False)
leaderboard

,Team,Win,Loss,Draw,Pts
11,Manchester City,12.0,2.0,5.0,41.0
12,Manchester Utd,12.0,3.0,4.0,40.0
9,Leicester City,12.0,5.0,2.0,38.0
18,West Ham,10.0,5.0,5.0,35.0
10,Liverpool,9.0,3.0,7.0,34.0
16,Tottenham,9.0,3.0,6.0,33.0
6,Everton,10.0,5.0,2.0,32.0
0,Arsenal,9.0,8.0,3.0,30.0
1,Aston Villa,9.0,7.0,2.0,29.0
4,Chelsea,8.0,6.0,5.0,29.0


Part 1. Predict without any stats

In [ ]:
#Change name to ID
results = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results[j][i]==Name[k]:
        results[j][i]=k
results

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
traindata=results.iloc[0:x,:]
preddata=results.iloc[x:380,:]
newresults=results
x_train = traindata[['Home', 'Away']]
y_train = traindata[['HScore', 'AScore']]
x_pred = preddata[['Home', 'Away']]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train, y_train)

In [ ]:
goals = model.predict(x_pred)

In [ ]:
for i in range(380-x):
  for j in range(1):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults[k][x+i]=goals[i][j]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.000000,3.0,1.8,0
1,5,0.7,1.000000,0.0,0.8,15
2,10,3.3,4.000000,3.0,0.6,8
3,18,1.1,0.000000,2.0,1.5,13
4,17,0.5,0.000000,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,0.333333,NaN,NaN,2
376,18,NaN,0.707107,NaN,NaN,15
377,14,NaN,1.242641,NaN,NaN,3
378,10,NaN,2.242641,NaN,NaN,5


In [ ]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults['HScore'][i]>newresults['AScore'][i]:
    leaderboard['Win'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=3
    leaderboard['Loss'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=0
  if newresults['HScore'][i]==newresults['AScore'][i]:
    leaderboard['Draw'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=1
    leaderboard['Draw'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=1
  if newresults['HScore'][i]<newresults['AScore'][i]:
    leaderboard['Loss'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=0
    leaderboard['Win'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=3

In [ ]:
leaderboard.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard

,Team,Win,Loss,Draw,Pts
11,Manchester City,21.0,5.0,5.0,68.0
12,Manchester Utd,17.0,8.0,5.0,56.0
10,Liverpool,16.0,6.0,8.0,56.0
16,Tottenham,15.0,8.0,7.0,52.0
0,Arsenal,15.0,11.0,4.0,49.0
6,Everton,15.0,10.0,4.0,49.0
8,Leeds United,15.0,12.0,3.0,48.0
9,Leicester City,15.0,12.0,3.0,48.0
18,West Ham,14.0,10.0,6.0,48.0
4,Chelsea,14.0,11.0,5.0,47.0


Part2: Using xG to predict the goal efficiency of each team in each match

In [ ]:
#Change name to ID
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,0.0,0.2,0.0,3.0,1.8,0.0,0
1,5,0.0,0.7,1.0,0.0,0.8,0.0,15
2,10,0.0,3.3,4.0,3.0,0.6,0.0,8
3,18,0.0,1.1,0.0,2.0,1.5,0.0,13
4,17,0.0,0.5,0.0,3.0,2.2,0.0,9
...,...,...,...,...,...,...,...,...
375,0,0.0,NaN,NaN,NaN,NaN,0.0,2
376,18,0.0,NaN,NaN,NaN,NaN,0.0,15
377,14,0.0,NaN,NaN,NaN,NaN,0.0,3
378,10,0.0,NaN,NaN,NaN,NaN,0.0,5


In [ ]:
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]


In [ ]:
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(380-x):
  results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
  results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.0,3.0,1.8,0.400000,0
1,5,0.300000,0.7,1.0,0.0,0.8,-1.600000,15
2,10,0.175000,3.3,4.0,3.0,0.6,0.800000,8
3,18,-2.200000,1.1,0.0,2.0,1.5,0.250000,13
4,17,-1.000000,0.5,0.0,3.0,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,-1.955635,NaN,NaN,NaN,NaN,-0.844115,2
376,18,-1.402910,NaN,NaN,NaN,NaN,-0.981102,15
377,14,-3.127062,NaN,NaN,NaN,NaN,-1.681089,3
378,10,-0.709130,NaN,NaN,NaN,NaN,-0.168639,5


In [ ]:
traindata2=results2.iloc[0:x,:]
preddata2=results2.iloc[x:380,:]
newresults2=results2
x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
y_train2 = traindata2[['HScore', 'AScore']]
x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]

In [ ]:
model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)

In [ ]:
goals2 = model2.predict(x_pred2)

In [ ]:
for i in range(380-x):
  for j in range(2):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults2[k][x+i]=goals2[i][j]
newresults2

In [ ]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults2['HScore'][i]>newresults2['AScore'][i]:
    leaderboard2['Win'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=3
    leaderboard2['Loss'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=0
  if newresults2['HScore'][i]==newresults2['AScore'][i]:
    leaderboard2['Draw'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=1
    leaderboard2['Draw'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=1
  if newresults2['HScore'][i]<newresults2['AScore'][i]:
    leaderboard2['Loss'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=0
    leaderboard2['Win'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=3

In [ ]:
leaderboard2.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard2

,Team,Win,Loss,Draw,Pts
12,Manchester Utd,24.0,10.0,4.0,76.0
11,Manchester City,23.0,8.0,7.0,76.0
9,Leicester City,22.0,12.0,4.0,70.0
15,Southampton,21.0,12.0,5.0,68.0
10,Liverpool,20.0,10.0,8.0,68.0
0,Arsenal,21.0,14.0,3.0,66.0
6,Everton,21.0,14.0,3.0,66.0
4,Chelsea,20.0,13.0,5.0,65.0
16,Tottenham,18.0,14.0,6.0,60.0
1,Aston Villa,17.0,19.0,2.0,53.0


Repeat Part 2 1000 times to find the expected win percentage

In [ ]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]  
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for a in range(y):
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard2.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

TypeError: ignored

Def eff

In [ ]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(2,'HDefEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2.insert(6,'ADefEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
DefEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]   
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(x):#Def
  if results2['AScore'][i]==0:#Total
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/0.5
  else:
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]
  DefEff[2][results2['Home'][i]]+=1
  if results2['HScore'][i]==0:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/0.5
  else:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/results2['HScore'][i]
  DefEff[2][results2['Away'][i]]+=1
for i in range(20):#Mean
  DefEff[3][i]=DefEff[1][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:#Var
    DefEff[4][results2['Home'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/0.5-DefEff[3][results2['Away'][i]])**2*0.5
  else:
    DefEff[4][results2['Home'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]-DefEff[3][results2['Away'][i]])**2*0.5
  if results2['HScore'][i]==0:
    DefEff[4][results2['Away'][i]]+=((results2['HxG'][i]-results2['HScore'][i])/0.5-DefEff[3][results2['Home'][i]])**2*0.5
  else:
    DefEff[4][results2['Away'][i]]+=((results2['HxG'][i]-results2['HScore'][i])/results2['HScore'][i]-DefEff[3][results2['Home'][i]])**2*0.5
for i in range(20):
  DefEff[4][i]=DefEff[4][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:
    results2['HDefEff'][i]=(results2['AxG'][i]-results2['AScore'][i])/0.5*0.5
  else:
    results2['HDefEff'][i]=(results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]*0.5
  if results2['HScore'][i]==0:
    results2['ADefEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5*0.5
  else:
    results2['ADefEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]*0.5
for a in range(y):#model
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
      results2['HDefEff'][x+i]=np.random.normal(DefEff[3][results2['Home'][x+i]],DefEff[4][results2['Home'][x+i]])*0.5
      results2['ADefEff'][x+i]=np.random.normal(DefEff[3][results2['Away'][x+i]],DefEff[4][results2['Away'][x+i]])*0.5
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard2.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%,Top4,Top4%,Rel,Rel%
11,Manchester City,577.0,57.7,990.0,99.0,0.0,0.0
12,Manchester Utd,194.0,19.4,904.0,90.4,0.0,0.0
10,Liverpool,178.0,17.8,834.0,83.4,0.0,0.0
9,Leicester City,37.0,3.7,477.0,47.7,0.0,0.0
6,Everton,7.0,0.7,215.0,21.5,0.0,0.0
0,Arsenal,4.0,0.4,242.0,24.2,0.0,0.0
4,Chelsea,1.0,0.1,100.0,10.0,0.0,0.0
15,Southampton,1.0,0.1,71.0,7.1,0.0,0.0
1,Aston Villa,1.0,0.1,39.0,3.9,1.0,0.1
16,Tottenham,0.0,0.0,110.0,11.0,0.0,0.0


In [ ]:
y=1000
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(2,'HDefEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2.insert(6,'ADefEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
DefEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]   
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(x):#Def
  if results2['AScore'][i]==0:#Total
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/0.5
  else:
    DefEff[1][results2['Home'][i]]+=(results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]
  DefEff[2][results2['Home'][i]]+=1
  if results2['HScore'][i]==0:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/0.5
  else:
    DefEff[1][results2['Away'][i]]+=(results2['HxG'][i]-results2['HScore'][i])/results2['HScore'][i]
  DefEff[2][results2['Away'][i]]+=1
for i in range(20):#Mean
  DefEff[3][i]=DefEff[1][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:#Var
    DefEff[4][results2['Home'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/0.5-DefEff[3][results2['Away'][i]])**2*0.5
  else:
    DefEff[4][results2['Home'][i]]+=((results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]-DefEff[3][results2['Away'][i]])**2*0.5
  if results2['HScore'][i]==0:
    DefEff[4][results2['Away'][i]]+=((results2['HxG'][i]-results2['HScore'][i])/0.5-DefEff[3][results2['Home'][i]])**2*0.5
  else:
    DefEff[4][results2['Away'][i]]+=((results2['HxG'][i]-results2['HScore'][i])/results2['HScore'][i]-DefEff[3][results2['Home'][i]])**2*0.5
for i in range(20):
  DefEff[4][i]=DefEff[4][i]/DefEff[2][i]
for i in range(x):
  if results2['AScore'][i]==0:
    results2['HDefEff'][i]=(results2['AxG'][i]-results2['AScore'][i])/0.5*0.5
  else:
    results2['HDefEff'][i]=(results2['AxG'][i]-results2['AScore'][i])/results2['AScore'][i]*0.5
  if results2['HScore'][i]==0:
    results2['ADefEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5*0.5
  else:
    results2['ADefEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]*0.5
for a in range(y):#model
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
      results2['HDefEff'][x+i]=np.random.normal(DefEff[3][results2['Home'][x+i]],DefEff[4][results2['Home'][x+i]])*0.5
      results2['ADefEff'][x+i]=np.random.normal(DefEff[3][results2['Away'][x+i]],DefEff[4][results2['Away'][x+i]])*0.5
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']].astype('float64').values.reshape(x,6,1)
    y_train2 = (traindata2[['HScore','AScore']]).astype('float64').values.reshape(x,1,2)
    x_pred2 = preddata2[['Home','HGoalEff','HDefEff','AGoalEff','ADefEff','Away']].astype('float64').values.reshape(380-x,6,1)
    model2 = tf.keras.Sequential([
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(128, activation = tf.nn.relu),
          tf.keras.layers.Dense(2, activation='linear')])
    model2.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy'])
    model2.fit(x_train2, y_train2,batch_size=18, epochs=10,verbose=0)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][0][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard2.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard2.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard    


,Team,Champ,Win%,Top4,Top4%,Rel,Rel%
12,Manchester Utd,446.0,44.6,773.0,77.3,0.0,0.0
11,Manchester City,311.0,31.1,815.0,81.5,0.0,0.0
1,Aston Villa,181.0,18.1,258.0,25.8,0.0,0.0
6,Everton,44.0,4.4,497.0,49.7,0.0,0.0
10,Liverpool,17.0,1.7,394.0,39.4,0.0,0.0
0,Arsenal,1.0,0.1,125.0,12.5,3.0,0.3
9,Leicester City,0.0,0.0,881.0,88.1,0.0,0.0
2,Brighton,0.0,0.0,89.0,8.9,309.0,30.9
4,Chelsea,0.0,0.0,72.0,7.2,0.0,0.0
3,Burnley,0.0,0.0,67.0,6.7,4.0,0.4


Covid data

In [85]:
y=1000
results3 = data2.iloc[:,1:7]
Name=['Atalanta','Benevento','Bologna','Cagliari','Crotone','Fiorentina','Genoa','Hellas Verona','Inter','Juventus','Lazio','Milan','Napoli','Parma','Roma','Sampdoria','Sassuolo','Spezia','Torino','Udinese']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results3[j][i]==Name[k]:
        results3[j][i]=k
results3.insert(1,'HGoalEff',np.zeros(380), True)
results3.insert(2,'HDefEff',np.zeros(380), True)
results3.insert(3,'HCovid',np.zeros(380), True)
results3.insert(6,'AGoalEff',np.zeros(380), True)
results3.insert(6,'ADefEff',np.zeros(380), True)
results3.insert(6,'ACovid',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
DefEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
aclb = {'Team': Name,
  'Champ':np.zeros(20),
  'Champ%':np.zeros(20),
  'Top4':np.zeros(20),
  'Top4%':np.zeros(20),
  'Rel':np.zeros(20),
  'Rel%':np.zeros(20)
    }
cov=[np.arange(20),np.zeros(20)]
covid=coviddata
for i in range(380):
  for j in range(2):
    if j==0:
      k='HCovid'
      l='Home'
    if j==1:
      k='ACovid'
      l='Away'
    results3[k][i]=covid[str(results3[l][i])][cov[1][results3[l][i]]]
    cov[1][results3[l][i]]+=1

for i in range(x):
  if results3['HScore'][i]==0:
    GoalEff[1][results3['Home'][i]]+=(results3['HScore'][i]-results3['HxG'][i])/0.5
  else:
    GoalEff[1][results3['Home'][i]]+=(results3['HScore'][i]-results3['HxG'][i])/results3['HScore'][i]
  GoalEff[2][results3['Home'][i]]+=1
  if results3['AScore'][i]==0:
    GoalEff[1][results3['Away'][i]]+=(results3['AScore'][i]-results3['AxG'][i])/0.5
  else:
    GoalEff[1][results3['Away'][i]]+=(results3['AScore'][i]-results3['AxG'][i])/results3['AScore'][i]
  GoalEff[2][results3['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results3['HScore'][i]==0:
    GoalEff[4][results3['Home'][i]]+=((results3['HScore'][i]-results3['HxG'][i])/0.5-GoalEff[3][results3['Home'][i]])**2
  else:
    GoalEff[4][results3['Home'][i]]+=((results3['HScore'][i]-results3['HxG'][i])/results3['HScore'][i]-GoalEff[3][results3['Home'][i]])**2
  if results3['AScore'][i]==0:
    GoalEff[4][results3['Away'][i]]+=((results3['AScore'][i]-results3['AxG'][i])/0.5-GoalEff[3][results3['Away'][i]])**2
  else:
    GoalEff[4][results3['Away'][i]]+=((results3['AScore'][i]-results3['AxG'][i])/results3['AScore'][i]-GoalEff[3][results3['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results3['HScore'][i]==0:
    results3['HGoalEff'][i]=(results3['HScore'][i]-results3['HxG'][i])/0.5
  else:
    results3['HGoalEff'][i]=(results3['HScore'][i]-results3['HxG'][i])/results3['HScore'][i]
  if results3['AScore'][i]==0:
    results3['AGoalEff'][i]=(results3['AScore'][i]-results3['AxG'][i])/0.5
  else:
    results3['AGoalEff'][i]=(results3['AScore'][i]-results3['AxG'][i])/results3['AScore'][i]
for i in range(x):#Def
  if results3['AScore'][i]==0:#Total
    DefEff[1][results3['Home'][i]]+=(results3['AxG'][i]-results3['AScore'][i])/0.5
  else:
    DefEff[1][results3['Home'][i]]+=(results3['AxG'][i]-results3['AScore'][i])/results3['AScore'][i]
  DefEff[2][results3['Home'][i]]+=1
  if results3['HScore'][i]==0:
    DefEff[1][results3['Away'][i]]+=(results3['HxG'][i]-results3['HScore'][i])/0.5
  else:
    DefEff[1][results3['Away'][i]]+=(results3['HxG'][i]-results3['HScore'][i])/results3['HScore'][i]
  DefEff[2][results3['Away'][i]]+=1
for i in range(20):#Mean
  DefEff[3][i]=DefEff[1][i]/DefEff[2][i]
for i in range(x):
  if results3['AScore'][i]==0:#Var
    DefEff[4][results3['Home'][i]]+=((results3['AxG'][i]-results3['AScore'][i])/0.5-DefEff[3][results3['Away'][i]])**2*0.5
  else:
    DefEff[4][results3['Home'][i]]+=((results3['AxG'][i]-results3['AScore'][i])/results3['AScore'][i]-DefEff[3][results3['Away'][i]])**2*0.5
  if results3['HScore'][i]==0:
    DefEff[4][results3['Away'][i]]+=((results3['HxG'][i]-results3['HScore'][i])/0.5-DefEff[3][results3['Home'][i]])**2*0.5
  else:
    DefEff[4][results3['Away'][i]]+=((results3['HxG'][i]-results3['HScore'][i])/results3['HScore'][i]-DefEff[3][results3['Home'][i]])**2*0.5
for i in range(20):
  DefEff[4][i]=DefEff[4][i]/DefEff[2][i]
for i in range(x):
  if results3['AScore'][i]==0:
    results3['HDefEff'][i]=(results3['AxG'][i]-results3['AScore'][i])/0.5*0.5
  else:
    results3['HDefEff'][i]=(results3['AxG'][i]-results3['AScore'][i])/results3['AScore'][i]*0.5
  if results3['HScore'][i]==0:
    results3['ADefEff'][i]=(results3['HScore'][i]-results3['HxG'][i])/0.5*0.5
  else:
    results3['ADefEff'][i]=(results3['HScore'][i]-results3['HxG'][i])/results3['HScore'][i]*0.5
for a in range(y):#model
    for i in range(380-x):
      results3['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results3['Home'][x+i]],GoalEff[4][results3['Home'][x+i]])
      results3['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results3['Away'][x+i]],GoalEff[4][results3['Away'][x+i]])
      results3['HDefEff'][x+i]=np.random.normal(DefEff[3][results3['Home'][x+i]],DefEff[4][results3['Home'][x+i]])*0.5
      results3['ADefEff'][x+i]=np.random.normal(DefEff[3][results3['Away'][x+i]],DefEff[4][results3['Away'][x+i]])*0.5
    traindata3=results3.iloc[0:x,:]
    preddata2=results3.iloc[x:380,:]
    newresults3=results3
    x_train3 = traindata3[['Home','HCovid','HGoalEff','HDefEff','AGoalEff','ADefEff','ACovid','Away']]
    y_train3 = traindata3[['HScore', 'AScore']]
    x_pred3 = preddata2[['Home','HCovid','HGoalEff','HDefEff','AGoalEff','ADefEff','ACovid','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train3, y_train3)
    goals2 = model2.predict(x_pred3)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults3[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard3 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults3['HScore'][i]>newresults3['AScore'][i]:
        leaderboard3['Win'][newresults3['Home'][i]]+=1
        leaderboard3['Pts'][newresults3['Home'][i]]+=3
        leaderboard3['Loss'][newresults3['Away'][i]]+=1
        leaderboard3['Pts'][newresults3['Away'][i]]+=0
      if newresults3['HScore'][i]==newresults3['AScore'][i]:
        leaderboard3['Draw'][newresults3['Home'][i]]+=1
        leaderboard3['Pts'][newresults3['Home'][i]]+=1
        leaderboard3['Draw'][newresults3['Away'][i]]+=1
        leaderboard3['Pts'][newresults3['Away'][i]]+=1
      if newresults3['HScore'][i]<newresults3['AScore'][i]:
        leaderboard3['Loss'][newresults3['Home'][i]]+=1
        leaderboard3['Pts'][newresults3['Home'][i]]+=0
        leaderboard3['Win'][newresults3['Away'][i]]+=1
        leaderboard3['Pts'][newresults3['Away'][i]]+=3
    c=leaderboard3.loc[leaderboard3['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
    for l in range(4):
      d=leaderboard3.nlargest(4,'Pts')['Team'].iloc[l]
      aclb['Top4'][d]+=1
    for l in range(3):
      d=leaderboard3.nsmallest(3,'Pts')['Team'].iloc[l]
      aclb['Rel'][d]+=1    
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%','Top4','Top4%','Rel','Rel%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard['Top4%']=accleaderboard['Top4']/y*100
accleaderboard['Rel%']=accleaderboard['Rel']/y*100
accleaderboard.sort_values(by=['Rel'],inplace=True, ascending=True)
accleaderboard.sort_values(by=['Champ','Top4'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%,Top4,Top4%,Rel,Rel%
8,Inter,599.0,59.9,998.0,99.8,0.0,0.0
9,Juventus,201.0,20.1,993.0,99.3,0.0,0.0
11,Milan,166.0,16.6,991.0,99.1,0.0,0.0
10,Lazio,34.0,3.4,955.0,95.5,0.0,0.0
14,Roma,0.0,0.0,56.0,5.6,0.0,0.0
12,Napoli,0.0,0.0,6.0,0.6,0.0,0.0
15,Sampdoria,0.0,0.0,1.0,0.1,0.0,0.0
0,Atalanta,0.0,0.0,0.0,0.0,0.0,0.0
16,Sassuolo,0.0,0.0,0.0,0.0,0.0,0.0
7,Hellas Verona,0.0,0.0,0.0,0.0,0.0,0.0
